# Demonstration of Trading P&L

A position consists of a number of buy or sell trades. When the
position is flat (when there are no trades or the quantity of buys equals the
quantity of sells) there is an unambiguous result for the P&L (the amount spent
minus the amount received). Up until this point the P&L depends on which buys
are matched with which sells, and which unmatched trades remain.

Typically, accountants prefer a FIFO (first in, first out) style of matching.
So if there are multiple buys, a sell matches against the earliest buy.

Traders sometimes prefer a "worst price" approach, were a sell is matched
against the highest price buy (the bu with the worst price).

Regardless of the approach the P&L can be characterized by the following
properties:

* quantity - how much of the asset is held.
* cost - how much has it cost to accrue the asset.
* realized - how much profit (or loss) was realized by selling from a long
  position, or buying from a short.
* unmatched - trades which have not yet been completely matched.

If the new trade extends the position (a buy from a long or flat position or a
sell from a flat or short position) the quantity increases by that of the trade
and also the cost.

If the trade reduces the position a matching trade must be found. Taking FIFO
as the method, the oldest trade is taken. There are three possibilities: The
matching trade might be exactly the same quantity (but of opposite sign), the
trade might have the larger quantity, or the match might have the larger quantity.
Where the quantities don't match exactly there must be a split. If the match
quantity is greater, the match is split and the spare is returned to the unmatched.
If the trade is larger it is split and the remainder becomes the next trade to
match.

Each match will "realize" P&L (the trade either made or lost money). The cost of
the position will also be changed as the cost of the matched trades are removed.

In [1]:
from decimal import Decimal
from typing import Literal

from jetblack_pnl.core import (
    add_trade,
    TradingPnl,
    IMatchedPool,
    IUnmatchedPool,
)
from jetblack_pnl.impl.simple import (
    MatchedPool,
    UnmatchedPool,
    Security,
    Trade
)

In [2]:
Side = Literal['buy', 'sell']

def trade(trade_id: int, side: Side, quantity: int, price: int) -> Trade:
    sign = 1 if side == 'buy' else -1
    return Trade(quantity * sign, price)

def display(pnl: TradingPnl[str], matched: IMatchedPool[None, None], unmatched: IUnmatchedPool[None, None]) -> None:
    avg_cost = -pnl.cost / pnl.quantity if pnl.quantity != 0 else 0
    text = f"pnl: {pnl}, avg_cost: {avg_cost}" + "\n"
    text += "matched:\n"
    if not matched.pool(None):
        text += "    None\n"
    else:
        for opening, closing in matched.pool(None):
            text += f"    ({opening}) matches ({closing})" + "\n"
    text += "unmatched:\n"
    if not unmatched.pool(None):
        text += "    None\n"
    else:
        for trade in unmatched.pool(None):
            text += f"    " + str(trade) + "\n"
    print(text)

In [3]:
sec = Security("AAPL", 1, False)
trade_id = 0
matched = MatchedPool()
unmatched = UnmatchedPool.Fifo()
pnl = TradingPnl[str](Decimal(0), Decimal(0), Decimal(0))
display(pnl, matched, unmatched)

pnl: position: 0, cost: 0, realized: 0, avg_cost: 0
matched:
    None
unmatched:
    None



In [4]:
trade_id += 1
pnl = add_trade(pnl, trade(trade_id, 'buy', 6, 100), sec, unmatched, matched, None)
display(pnl, matched, unmatched)

pnl: position: 6, cost: -600, realized: 0, avg_cost: 100
matched:
    None
unmatched:
    buy 6 of trade: buy 6 @ 100



In [5]:
trade_id += 1
pnl = add_trade(pnl, trade(trade_id, 'buy', 6, 106), sec, unmatched, matched, None)
display(pnl, matched, unmatched)

pnl: position: 12, cost: -1236, realized: 0, avg_cost: 103
matched:
    None
unmatched:
    buy 6 of trade: buy 6 @ 100
    buy 6 of trade: buy 6 @ 106



In [6]:
trade_id += 1
pnl = add_trade(pnl, trade(trade_id, 'buy', 6, 103), sec, unmatched, matched, None)
display(pnl, matched, unmatched)

pnl: position: 18, cost: -1854, realized: 0, avg_cost: 103
matched:
    None
unmatched:
    buy 6 of trade: buy 6 @ 100
    buy 6 of trade: buy 6 @ 106
    buy 6 of trade: buy 6 @ 103



In [7]:
trade_id += 1
pnl = add_trade(pnl, trade(trade_id, 'sell', 9, 105), sec, unmatched, matched, None)
display(pnl, matched, unmatched)

pnl: position: 9, cost: -936, realized: 27, avg_cost: 104
matched:
    (buy 6 of trade: buy 6 @ 100) matches (sell 6 of trade: sell 9 @ 105)
    (buy 3 of trade: buy 6 @ 106) matches (sell 3 of trade: sell 9 @ 105)
unmatched:
    buy 3 of trade: buy 6 @ 106
    buy 6 of trade: buy 6 @ 103



In [8]:
trade_id += 1
pnl = add_trade(pnl, trade(trade_id, 'sell', 12, 107), sec, unmatched, matched, None)
display(pnl, matched, unmatched)

pnl: position: -3, cost: 321, realized: 54, avg_cost: 107
matched:
    (buy 6 of trade: buy 6 @ 100) matches (sell 6 of trade: sell 9 @ 105)
    (buy 3 of trade: buy 6 @ 106) matches (sell 3 of trade: sell 9 @ 105)
    (buy 3 of trade: buy 6 @ 106) matches (sell 3 of trade: sell 12 @ 107)
    (buy 6 of trade: buy 6 @ 103) matches (sell 6 of trade: sell 12 @ 107)
unmatched:
    sell 3 of trade: sell 12 @ 107



In [9]:
trade_id += 1
pnl = add_trade(pnl, trade(trade_id, 'buy', 3, 105), sec, unmatched, matched, None)
display(pnl, matched, unmatched)

pnl: position: 0, cost: 0, realized: 60, avg_cost: 0
matched:
    (buy 6 of trade: buy 6 @ 100) matches (sell 6 of trade: sell 9 @ 105)
    (buy 3 of trade: buy 6 @ 106) matches (sell 3 of trade: sell 9 @ 105)
    (buy 3 of trade: buy 6 @ 106) matches (sell 3 of trade: sell 12 @ 107)
    (buy 6 of trade: buy 6 @ 103) matches (sell 6 of trade: sell 12 @ 107)
    (sell 3 of trade: sell 12 @ 107) matches (buy 3 of trade: buy 3 @ 105)
unmatched:
    None

